In [1]:
# Import Qiskit and qiskit-juqcs

import qiskit
from juqcs.provider import JuqcsProvider

Credentials are valid! You may start using JUQCS now.


In [2]:
import qiskit.tools.jupyter
%qiskit_version_table


In [3]:
! pip show qiskit-juqcs

Name: qiskit-juqcs
Version: 0.8.0
Summary: Qiskit provider for JUQCS (Juelich Universal Quantum Computer Simulator)
Home-page: https://jugit.fz-juelich.de/qip/juniq-platform/qiskit-juqcs
Author: Carlos Gonzalez, Cong Luo
Author-email: c.gonzalez.calaza@fz-juelich.de, c.luo@fz-juelich.de
License: MIT
Location: /p/software/hdfcloud/stages/2023/software/Qiskit-juqcs/0.8.0-gpsmkl-2022a/lib/python3.10/site-packages
Requires: numpy, python-dateutil, pyunicore, qiskit
Required-by: 


In [4]:
# Create a quantum circuit

nqubits = 4
circuit = qiskit.QuantumCircuit(nqubits)
circuit.h(0)
for i in range(1,nqubits): 
    circuit.cx(i-1,i)
circuit.measure_all()
print(circuit)

        ┌───┐                ░ ┌─┐         
   q_0: ┤ H ├──■─────────────░─┤M├─────────
        └───┘┌─┴─┐           ░ └╥┘┌─┐      
   q_1: ─────┤ X ├──■────────░──╫─┤M├──────
             └───┘┌─┴─┐      ░  ║ └╥┘┌─┐   
   q_2: ──────────┤ X ├──■───░──╫──╫─┤M├───
                  └───┘┌─┴─┐ ░  ║  ║ └╥┘┌─┐
   q_3: ───────────────┤ X ├─░──╫──╫──╫─┤M├
                       └───┘ ░  ║  ║  ║ └╥┘
meas: 4/════════════════════════╩══╩══╩══╩═
                                0  1  2  3 


In [5]:
# Instantiate the JUQCS Provider and backend for submitting the quantum circuit
provider = JuqcsProvider()

#  Select one of the backends by name:
# ----------------------------------------------------------------------------------------------------
# backend name               | description and specifications
# ----------------------------------------------------------------------------------------------------
# 'qasm_simulator'           | CPU version of qasm_simulator, support up to 40 qubits with 10000 shots
# 'qasm_simulator_gpu'       | GPU version of qasm_simulator, support up to 37 qubits with 10000 shots
# 'statevector_simulator'    | CPU version of statevector_simulator, support up to 20 qubits
# 'statevector_simulator_gpu'| GPU version of statevector_simulator, support up to 20 qubits
# ----------------------------------------------------------------------------------------------------

# use qasm_simulator as an example
backend = provider.get_backend('qasm_simulator')

In [6]:
# Allocate resources on JURECA-DC_cpu for a given time, proportional to the nr. of qubits in the circuit

print('ALLOCATING...')
backend.allocate(minutes=30, max_qubits=nqubits) #, reservation='juniq-test')

ALLOCATING...


Please choose one of the following projects to charge the allocation to: ['jqctst02', 'jqctst01']. Get rid of this prompt by actively passing a project string to the "allocate" method. jqctst02


Trying to allocate compute resources under project jqctst02, this may take a few minutes... (please do not abort, otherwise the allocation may be lost and the compute time wasted).
Resource allocation #12341140 of 4 qubits available until 2023-10-05 16:51:25.


In [7]:
# Submit quantum circuit for execution on the allocated resources

print('SUBMITTING CIRCUITS...')
job = qiskit.execute(circuit, backend, shots=1000, seed=10)
job.status()

# Note: 
# Currently (12.2023), because of a known issue in job sheduling system in Jureca DC, 
# if the backend is "qasm_simulator" and the number of qubits is more than 34, only 
# one circuit can be submitted for each allocation.

SUBMITTING CIRCUITS...
Submitting circuits for simulation, this may take a few minutes...


<JobStatus.DONE: 'job has successfully run'>

In [8]:
# Show quantum circuit simulation results

from pprint import pprint
result = job.result()
pprint(result.to_dict())

# Note: 
# For jobs that would require longer time for computational resources allocation, if 
# the user retrieves results after reconnecting to JUNIQ, it is recommended to save 
# the results to the file system of the Jupter environment for further loading, below
# we provide one of the possibilities:
# -------------------------------------------
#   import json
#   result = job.result()
#   with open("results", "w") as fp:
#       json.dump(result.to_dict(), fp)

{'backend_name': 'qasm_simulator',
 'backend_version': '0.1.0',
 'date': '2023-10-05 16:24:24.379884',
 'header': None,
 'job_id': '9dbcb007-4c34-4cd8-b495-d0b42775d6d4',
 'qobj_id': None,
 'results': [{'data': {'counts': {'0x0': 510, '0xf': 490}},
              'header': {'clbit_labels': [['meas', 0],
                                          ['meas', 1],
                                          ['meas', 2],
                                          ['meas', 3]],
                         'creg_sizes': [['meas', 4]],
                         'global_phase': 0,
                         'memory_slots': 4,
                         'n_qubits': 4,
                         'name': 'circuit-96',
                         'qreg_sizes': [['q', 4]],
                         'qubit_labels': [['q', 0],
                                          ['q', 1],
                                          ['q', 2],
                                          ['q', 3]]},
              'meas_level': 2,
         

In [9]:
# Release the allocated resources so you don't get charged for what you don't use
print('DEALLOCATING...')
backend.deallocate()

DEALLOCATING...
Trying to deallocate compute resources, this may take a few minutes...
Allocation #12341140 revoked.
